# Portland Pizza Tour

## data clean up

In [2]:
df = pd.read_csv("Portland Pizza Tour.csv")
df.head()

,Unnamed: 0,Pizza,Variety,Taste,Presentation,Crust,Sauce,Toppings,Creativity,Message,Availability (did they have what you wanted),sides,leftovers,total,Notes
0,Alex,Handsome Pizza,marinara,8.0,7.0,8.0,8.5,6.0,6.0,10.0,7.0,5.0,5.0,7.05,"not the best for vegan options. very tangy, bu..."
1,Alice,NaN,sergio olmos,7.0,7.0,8.0,NaN,8.0,8.0,9.0,7.0,5.0,3.0,6.89,no comment I guess
2,Alex,Mississippi pizza,amaizeing,8.0,8.0,7.0,6.0,9.0,9.0,NaN,9.0,8.0,8.0,8,great vegan options. caramelized onion sauce w...
3,Alice,NaN,spicoli,7.0,8.0,8.0,6.0,8.0,7.0,NaN,9.0,9.0,7.0,7.666666667,the salad was the highlight of this meal. it h...
4,Alex,Gracie's Apizza,vegan,9.0,9.0,9.0,9.0,9.0,9.0,NaN,9.0,6.0,NaN,8.625,a lot of things are right about this pizza. th...


In [3]:
import numpy as np

df.rename(
    columns={
        "Unnamed: 0": "Name",
        "Pizza": "Vendor",
        "Availability (did they have what you wanted)": "Availability",
    },
    inplace=True,
)
df.total.replace("#DIV/0!", np.nan, inplace=True)

In [4]:
df.Vendor.fillna(method="ffill", inplace=True)
df.head()

,Name,Vendor,Variety,Taste,Presentation,Crust,Sauce,Toppings,Creativity,Message,Availability,sides,leftovers,total,Notes
0,Alex,Handsome Pizza,marinara,8.0,7.0,8.0,8.5,6.0,6.0,10.0,7.0,5.0,5.0,7.05,"not the best for vegan options. very tangy, bu..."
1,Alice,Handsome Pizza,sergio olmos,7.0,7.0,8.0,NaN,8.0,8.0,9.0,7.0,5.0,3.0,6.89,no comment I guess
2,Alex,Mississippi pizza,amaizeing,8.0,8.0,7.0,6.0,9.0,9.0,NaN,9.0,8.0,8.0,8,great vegan options. caramelized onion sauce w...
3,Alice,Mississippi pizza,spicoli,7.0,8.0,8.0,6.0,8.0,7.0,NaN,9.0,9.0,7.0,7.666666667,the salad was the highlight of this meal. it h...
4,Alex,Gracie's Apizza,vegan,9.0,9.0,9.0,9.0,9.0,9.0,NaN,9.0,6.0,NaN,8.625,a lot of things are right about this pizza. th...


In [5]:
df.drop(df[df.total.isna()].index, inplace=True)
df.shape

(48, 15)

In [6]:
df["Name"].value_counts()

Name
Alex     25
Alice    23
Name: count, dtype: int64

In [15]:
df.isna().sum()

Name             0
Vendor           0
Variety          7
Taste            0
Presentation     0
Crust            0
Sauce            1
Toppings         0
Creativity       0
Message         34
Availability     0
sides           32
leftovers       34
total            0
Notes           19
dtype: int64

In [18]:
df.columns

Index(['Name', 'Vendor', 'Variety', 'Taste', 'Presentation', 'Crust', 'Sauce',
       'Toppings', 'Creativity', 'Message', 'Availability', 'sides',
       'leftovers', 'total', 'Notes'],
      dtype='object')

In [20]:
score_cols = [
    "Taste",
    "Presentation",
    "Crust",
    "Sauce",
    "Toppings",
    "Creativity",
    "Message",
    "Availability",
    "sides",
    "leftovers",
]
len(score_cols)

10

In [22]:
# what is the maximum number of missing score values in a row?
df[score_cols].T.isna().sum().max()

3

currenty rankings with NaN values in the score are not being counted towards the total score, so more NaN values inflates the score. Need to correct for this. Instead we'll take the top 4 scores combined with the bottom 3 scores (total of 7 so in the worst case all scores will be populated) and average those

In [54]:
def get_final_score(row):
    """Calculate the final score for a row of pizza scores."""
    scores = row.dropna().sort_values(ascending=False).values
    return np.mean(np.concatenate([scores[4:], scores[-3:]]))

In [55]:
final_scores = []
for _, row in df[score_cols].iterrows():
    final_scores.append(get_final_score(row))

df["final_score"] = final_scores

In [57]:
df.head()

,Name,Vendor,Variety,Taste,Presentation,Crust,Sauce,Toppings,Creativity,Message,Availability,sides,leftovers,total,Notes,final_score
0,Alex,Handsome Pizza,marinara,8.0,7.0,8.0,8.5,6.0,6.0,10.0,7.0,5.0,5.0,7.05,"not the best for vegan options. very tangy, bu...",5.777778
1,Alice,Handsome Pizza,sergio olmos,7.0,7.0,8.0,NaN,8.0,8.0,9.0,7.0,5.0,3.0,6.89,no comment I guess,5.500000
2,Alex,Mississippi pizza,amaizeing,8.0,8.0,7.0,6.0,9.0,9.0,NaN,9.0,8.0,8.0,8,great vegan options. caramelized onion sauce w...,7.250000
3,Alice,Mississippi pizza,spicoli,7.0,8.0,8.0,6.0,8.0,7.0,NaN,9.0,9.0,7.0,7.666666667,the salad was the highlight of this meal. it h...,6.875000
4,Alex,Gracie's Apizza,vegan,9.0,9.0,9.0,9.0,9.0,9.0,NaN,9.0,6.0,NaN,8.625,a lot of things are right about this pizza. th...,8.142857


In [59]:
# sort by person then by final score
df.sort_values(["Name", "final_score"], ascending=False, inplace=True)

In [60]:
alex = df[df["Name"] == "Alex"].copy()
alice = df[df["Name"] == "Alice"].copy()

## Top 5 for Vegan Options

In [61]:
alex.head()

,Name,Vendor,Variety,Taste,Presentation,Crust,Sauce,Toppings,Creativity,Message,Availability,sides,leftovers,total,Notes,final_score
18,Alex,Pizzeria Otto,supremo vegan,10.0,10.0,10.0,10.0,10.0,9.0,9.0,9.0,NaN,NaN,9.625,the vegan cashew ricotta is the best I've ever...,9.142857
28,Alex,Pizza Doughnais (sourdough),tomato + vegan pep,10.0,8.0,10.0,10.0,9.0,9.0,8.0,9.0,NaN,9.0,9.111111111,NaN,8.500000
14,Alex,Red Sauce Pizza,it's me nummy,10.0,8.0,9.0,9.0,10.0,8.0,9.0,9.0,NaN,NaN,9,this pizza really hit the spot. a classic comb...,8.428571
26,Alex,Scottie's Pizza Parlor,vegan hawaii 505,10.0,10.0,10.0,10.0,8.0,8.0,NaN,9.0,NaN,9.0,9.25,NaN,8.428571
50,Alex,Fino Bistro,"Vegan (kalamata olives, roasted garlic, pineap...",10.0,9.0,9.0,10.0,10.0,9.0,NaN,8.0,8.0,8.0,9,Incredible roasted red pepper base!,8.250000


## Top 5 for ~~murderers~~ meat eaters

In [62]:
alice.head()

,Name,Vendor,Variety,Taste,Presentation,Crust,Sauce,Toppings,Creativity,Message,Availability,sides,leftovers,total,Notes,final_score
19,Alice,Pizzeria Otto,"Buffalo Moz w/pepperoni, garlic and wild mushr...",10.0,10.0,8.0,10.0,10.0,9.0,9.0,10.0,NaN,NaN,9.5,The inner crust is the shining star. I have ne...,8.857143
37,Alice,Dove Vivi,med olive,10.0,9.0,10.0,8.0,10.0,9.0,NaN,10.0,NaN,NaN,9.428571429,very very interesting. only complaint is that ...,8.666667
7,Alice,Ranch Pizza,special / pepperoni,9.0,9.0,10.0,8.0,8.0,8.0,NaN,9.0,NaN,9.0,8.75,The pan style crust blew me away. reminded me ...,8.142857
11,Alice,Lovelysfiftyfifty,chanterelle,10.0,9.0,9.0,9.0,9.0,10.0,NaN,8.0,7.0,NaN,8.875,"small selection, however they did have everyth...",8.142857
9,Alice,Tastebud,pesto,9.0,7.0,8.0,9.0,8.0,9.0,9.0,8.0,9.0,9.0,8.5,"I normally really enjoy this place, but today ...",8.000000


In [63]:
alex.to_csv("alex.csv", index=False)
alice.to_csv("alice.csv", index=False)